### This notebook consist of code for creating the html files for the website each time data is updated.

### Set-up

Importing classes

In [1]:
import json
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from jinja2.exceptions import UndefinedError
from pathlib import Path

Defining paths

In [2]:
GROUP_DATA_DIR = Path("../../group-data").resolve()
MEMBERS_DIR_PATH = GROUP_DATA_DIR / "members/"
WEBSITE_DATA_PATH = GROUP_DATA_DIR / "website_data/"
CONTENT_DIR_PATH = WEBSITE_DATA_PATH / "content"
TEMPLATE_DIR_PATH = GROUP_DATA_DIR.parent / "groupwebsite_generator" / "templates"
HOSTING_PATH = GROUP_DATA_DIR.parent / "kerzendorf-group.github.io"

Data Processing Parameters

In [3]:
# List of JSON files to be processed
JSON_FILES_TO_LOAD = [
    "general",
    "homepage",
    "research",
    "support",
    "contact",
]

# Needed columns for homepage
ARTICLE_METADATA_FIELDS = [
    "article_id",
    "category",
    "date",
    "tags",
    "title",
    "cover_image",
    "short_description"
]
# Groups and institution used in filtering data
GROUP_FILTER = ["DTI", "TARDIS", "ICER", "kerzendorf"]
INSTITUTION_FILTER = "Michigan State University"

# Map roles to standardized roles for consistency
ROLE_MAP = {
    "Assistant Professor": "Professor",
    "Professor": "Professor",
    "Visualization Consultant": "Visualization Consultant",
    "Research Consultant": "Research Consultant",
    "Research Software Engineer": "Research Software Engineer",
    "Professorial Assistant": "Undergraduate",
    "Visiting Researcher": "Postdoctoral Researcher",
    "Postdoctoral Researcher": "Postdoctoral Researcher",
}

#ROLE HIERARCHY for sorting the member list by role
ROLE_HIERARCHY= {
    'Professor': 1,
    'Postdoctoral Researcher': 2,
    'Graduate Student': 3,
    'Undergraduate Student': 4,
    'Undergraduate': 4,
    'Research Consultant': 5,
    'Research Assistant': 5,
    'Research Software Engineer': 5,
    'Visualization Consultant': 5
}

# Map degrees to standardized academic levels
DEGREE_MAP = {
    "Masters": "Graduate Student",
    "PhD": "Postdoctorate",  #  if end_date is present
    "Bachelors": "Graduate Student",
}

INDIVIDUAL_MEMBER_SECTION_MAP = {
    "education": "Education",
    "experiences": "Experience",
    "projects": "Projects",
    "awards": "Awards & Recognition",
    "outreach": "Outreach Programs",
}

Function to create proper HTML file names by replacing spaces with underscores

In [4]:
def page_link(a):
    """Return the HTML file name after replacing blank spaces(" ") with underscores("-")"""
    return a.replace(" ", "_") if " " in a else a

 Creating an instance of the Environment class that looks for templates. Page_link is set to the global variable so that it can be accessed by all templates

In [5]:
environment = Environment(
    loader=FileSystemLoader(TEMPLATE_DIR_PATH), extensions=["jinja2.ext.loopcontrols", "jinja2.ext.do"]
)
environment.globals["page_link"] = page_link

Function for looping through JSON files and loading their content into the 'data' dictionary 

In [6]:
def loading_json_files(json_data_list):
    """
    Load data from JSON files specified in a list of file names.

    Parameters:
    ----------
    json_data_list : list of str
        A list of file names (without extension) to load as JSON.

    Returns:
    -------
    dict
        A dictionary where keys are file names and values are the corresponding JSON data.

    Raises:
    ------
    FileNotFoundError:
        If a specified file does not exist.
    json.JSONDecodeError:
        If there's an issue decoding the JSON content from a file.

    """
    loaded_data = {}
    for data_id in json_data_list:
        data_path = WEBSITE_DATA_PATH / f"{data_id}.json"

        try:
            with open(data_path, "r") as json_file:
                loaded_data[data_id] = json.load(json_file)
        except FileNotFoundError:
            print(f"File '{json_file}.json' not found.")
        except json.JSONDecodeError:
            print(f"Error decoding JSON in '{json_file}.json'.")
    return loaded_data

Function to create a page

In [7]:
def create_page(template, html, **kwargs):
    """
    Create an HTML page using a Jinja2 template and save it to a specified path.

    Parameters:
    ----------
    template : str
        The filename of the Jinja2 template to be used.
    html : str
        The filename of the HTML file to be generated.
    **kwargs : dict
        Additional keyword arguments to be passed to the Jinja2 template for rendering.

    Returns:
    -------
    None

    """
    page_template = environment.get_template(template)
    template_level = html.count("/")
    page_html_path = HOSTING_PATH / html
    page_content = page_template.render(TEMPLATE_LEVEL=template_level, **kwargs)
    with open(page_html_path, mode="w", encoding="utf-8") as page:
        page.write(page_content)

Creating dataframes for articles which can be updated further 

In [8]:
def load_content_from_files(columns):
    """
    Load content data from JSON files into a Pandas DataFrame.

    Parameters
    ----------
    columns : list of str
        A list of column names to extract from the JSON files.

    Returns
    -------
    pandas.DataFrame
        A DataFrame containing the specified columns from the loaded JSON files.

    """
    content_data = {col: [] for col in columns}

    for json_file in CONTENT_DIR_PATH.iterdir():
        if json_file.suffix == ".json":
            with open(json_file, "r") as file:
                info = json.load(file)
                # Only load those articles where display is True
                if info.get("display"):
                    for col in columns:
                        content_data[col].append(info.get(col))

    content_df = pd.DataFrame(content_data)
    content_df["date"] = pd.to_datetime(content_df["date"], format="%m-%d-%Y")
    return content_df

Extract the Latest Content for Each Category from a DataFrame

In [9]:
def get_latest_content_df(input_data):
    """
    Extract the latest content for each category from a DataFrame.

    Parameters
    ----------
    input_data : pandas.DataFrame
        The input DataFrame containing content information.

    Returns
    -------
    pandas.DataFrame
        A DataFrame containing the latest content for each category.

    """
    # Sort the entire DataFrame by "category" and "date" in descending order
    sorted_data = input_data.sort_values(
        by=["category", "date"], ascending=[True, False]
    )

    # Get the first row for each category using groupby and head
    latest_data = sorted_data.groupby("category").head(1).copy()
    latest_data["date"] = pd.to_datetime(
        latest_data["date"], format="%m-%d-%Y"
    )
    latest_data = latest_data.sort_values(by="date", ascending=False)

    return latest_data

### Processing List Of JSON files

In [10]:
# Function Call
data = loading_json_files(JSON_FILES_TO_LOAD)

### Homepage

Storing selected columns for Homepage only

In [11]:
content_df = load_content_from_files(ARTICLE_METADATA_FIELDS)
latest_content_df = get_latest_content_df(content_df)

In [12]:
create_page(
    "homepage.html.j2",
    "index.html",
    general=data["general"],
    homepage=data["homepage"],
    recent_content=latest_content_df.to_dict(orient="records"),
)

### People Page

Filtering based on group and institution

In [13]:
def filter_edu_exp_data(df, valid_groups,valid_institution):
    """
    Filter education and experience data based on specified criteria.

    Parameters
    ----------
    df : pandas.DataFrame
        The DataFrame containing education and experience data.
    valid_groups : list
        List of valid groups to include in the filtered data.
    valid_institution : str
        The valid institution to include in the filtered data.

    Returns
    -------
    pandas.DataFrame
        A filtered DataFrame containing only the rows that meet the specified criteria.
    """
    group_mask = False
    institution_mask = False

    # Check if 'group' column exists and update mask accordingly
    if "group" in df.columns:
        group_mask = df["group"].isin(valid_groups)

    # Check if 'institution' column exists and update mask accordingly
    if "institution" in df.columns:
        institution_mask = df["institution"] == valid_institution

    final_mask = group_mask | institution_mask
    return df[final_mask]

Function to load education data

In [14]:
def load_education_experience_data(directory):
    """
    Load education and experience data from JSON files, filter based on criteria, and perform preprocessing.

    Parameters
    ----------
    directory : str or pathlib.Path
        The directory path containing "experiences.json" and "education.json" files.

    Returns
    -------
    pandas.DataFrame
        A DataFrame containing filtered and preprocessed education and experience data.
    """
    filtered_records = []
    file_names = ["experiences.json", "education.json"]
    for file_name in file_names:
        file_path = directory / file_name
        if file_path.exists():
            # Reading JSON data directly into a DataFrame
            records = pd.read_json(file_path)

            # filtering based on group and institution
            valid_records = filter_edu_exp_data(records, GROUP_FILTER, INSTITUTION_FILTER)

            filtered_records.append(valid_records)
        else:
            print(f"{file_path} does not exist")

    if filtered_records:
        combined_records = pd.concat(filtered_records, ignore_index=True)
    else:
        combined_records = pd.DataFrame()

    # if start_date column exists, fill with NaN if it doesn't
    if "start_date" not in combined_records:
        combined_records["start_date"] = pd.NaT

    # Convert start_date to datetime format
    combined_records["start_date"] = pd.to_datetime(combined_records["start_date"], errors="coerce")

    # Sort the DataFrame based on start_date
    combined_records = combined_records.sort_values(by="start_date", ascending=False)
    return combined_records

Function load social links directly

In [15]:
def load_social_links(social_dir):
    """
    Load social links from a JSON file.

    Parameters
    ----------
    social_dir : str or pathlib.Path
        The directory path containing the "social_links.json" file.

    Returns
    -------
    dict or None
        A dictionary containing social links or None if the file doesn't exist.
    """
    social_links = None
    social_links_file_path = social_dir / "social_links.json"
    if social_links_file_path.exists():
        with open(social_links_file_path, "r") as f:
            social_links = json.load(f)
    return social_links

Function to load topmost project title

In [16]:
def load_latest_project_title(project_dir):
    """
    Load the title of the topmost project from a JSON file.

    Parameters
    ----------
    project_dir : str or pathlib.Path
        The directory path containing the "projects.json" file.

    Returns
    -------
    str or None
        The title of the topmost project or None if the file doesn't exist or is empty.
    """
    projects_file_path = project_dir / "projects.json"
    topmost_project_title = None
    if projects_file_path.exists():
        projects_df = pd.read_json(projects_file_path)
        if not projects_df.empty:
            # Fetching the project title from the first row of the DataFrame
            topmost_project_title = projects_df.iloc[0].get("project_title")
    return topmost_project_title

Funtion to parse member data

In [17]:
def parse_member_data(member_dir):
    """
    Parse member-related data from JSON files in the specified directory.

    Parameters
    ----------
    member_dir : str or pathlib.Path
        The directory path containing member-related JSON files.

    Returns
    -------
    tuple
        A tuple containing education and experience DataFrame, social links dictionary,
        and the title of the current project.
    """
    member_json_dir = member_dir / "jsons"
    education_experience_df = load_education_experience_data(member_json_dir)
    current_project_title = load_latest_project_title(member_json_dir)
    social_links = load_social_links(member_json_dir)

    return education_experience_df, social_links, current_project_title

In [18]:
# Function to extract academic roles from education and experience data
def extract_member_academic_role(education_experience_df):
    """
    Extract the current academic role of a member based on education and experience data.

    Parameters
    ----------
    education_experience_df : pandas.DataFrame
        DataFrame containing education and experience data.

    Returns
    -------
    Tuple[str, bool]
        A tuple containing:
        - str: The current academic role of the member.
        - bool: True if the member is currently active, False otherwise.
    """

    # Check if these columns exist in dataframe
    for column in ["end_date", "group", "institution"]:
        if column not in education_experience_df.columns:
            education_experience_df[column] = None

    current_academic_role = None

    for _, row in education_experience_df.iterrows():
        role = row.get("role", None)
        degree = row.get("degree", None)

        if not current_academic_role:
            current_academic_role = ROLE_MAP.get(role, "")

            if degree == "PhD" and pd.isna(row["end_date"]):
                current_academic_role = "Graduate Student"  # if end_date is NaN
            elif degree == "Bachelors" and pd.isna(row["end_date"]):
                current_academic_role = "Undergraduate Student"
            elif not current_academic_role and degree in DEGREE_MAP:
                current_academic_role = DEGREE_MAP[degree]

    # Check for end dates outside the loop
    has_end_date = all(
        not pd.isna(date) for date in education_experience_df["end_date"]
    )
    is_current_member = not has_end_date

    return current_academic_role, is_current_member

In [19]:
def custom_role_sort(roles):
    """
    Sorts a list of roles based on academic role hierarchy and then by name.

    Parameters
    ----------
    roles : list[dict]
        List of dictionaries representing roles. Each dictionary should contain at least the keys 'academic_role' and 'name'.

    Returns
    -------
    list[dict]
        A sorted list of roles based on the academic role hierarchy and, in case of ties, sorted by name in ascending order.
    """
    sorted_roles = sorted(roles, key=lambda x: (ROLE_HIERARCHY.get(x['academic_role'], float('inf')), x['name']))
    return sorted_roles

In [20]:
# Lists to store data for current and alumni members
def fetch_member_data():
    """
    Fetch and process member data from directories in the specified members' directory.

    Returns
    -------
    Tuple[list, list]
        A tuple containing two lists:
        1. List of dictionaries representing current members' data.
        2. List of dictionaries representing alumni members' data.

    """
    current_people_page_list = []
    alumni_people_page_list = []
    # Looping through member directories to fetch and process member data
    for member_dir in MEMBERS_DIR_PATH.glob("*"):
        if not (member_info_fname := member_dir / "info.json").exists():
            continue
        with open(member_info_fname, "r") as file:
            member_info = json.load(file)
        (
            education_experience_df,
            social_links,
            current_project_title,
        ) = parse_member_data(member_dir)
        current_academic_role, is_current_member = extract_member_academic_role(
            education_experience_df
        )
        first_name = member_info["first_name"]
        last_name = member_info["last_name"]
        nick_name = member_info.get("nick_name")
        member_id = member_info["id"]
        image_path = member_info["image_path"]
        cover_image_path = member_info["cover_image_path"]

        name = f"{nick_name if nick_name else first_name} {last_name}"
        member_data = {
            "name": name,
            "academic_role": current_academic_role,
            "id": member_id,
            "current_project_title": current_project_title,
            "image_path": image_path,
            "cover_image_path": cover_image_path,
        }

        if social_links is not None:
            member_data["social_links"] = social_links
            
        if is_current_member:
            current_people_page_list.append(member_data)
        else:
            alumni_people_page_list.append(member_data)

    # # Sort current members by role
    current_people_page_list = custom_role_sort(current_people_page_list)
    
    # # Sort alumni members by role
    alumni_people_page_list = custom_role_sort(alumni_people_page_list)
    return current_people_page_list, alumni_people_page_list

In [21]:
current_people_page_list, alumni_people_page_list = fetch_member_data()
all_people_page_list = current_people_page_list + alumni_people_page_list
all_people_data = {person["id"]: person for person in all_people_page_list}

/home/abhinav/workspace/code/tardis-main/kerzendorf-group/group-data/members/jack_o_brien/jsons/experiences.json does not exist
/home/abhinav/workspace/code/tardis-main/kerzendorf-group/group-data/members/hayden_monk/jsons/experiences.json does not exist
/home/abhinav/workspace/code/tardis-main/kerzendorf-group/group-data/members/vicente_amado/jsons/experiences.json does not exist
/home/abhinav/workspace/code/tardis-main/kerzendorf-group/group-data/members/yuki_matsumura/jsons/experiences.json does not exist
/home/abhinav/workspace/code/tardis-main/kerzendorf-group/group-data/members/sona_chitchyan/jsons/education.json does not exist
/home/abhinav/workspace/code/tardis-main/kerzendorf-group/group-data/members/alexander_grunewald/jsons/experiences.json does not exist


In [22]:
create_page(
    "people.html.j2",
    "People.html",
    general=data["general"],
    current_members=current_people_page_list,
    alumni_members=alumni_people_page_list,
)

Current Members Page

In [23]:
create_page(
    "current_members.html.j2",
    "current_members.html",
    general=data["general"],
    current_members=current_people_page_list,
)

Alumni Members Page

In [24]:
create_page(
    "alumni_members.html.j2",
    "alumni_members.html",
    general=data["general"],
    alumni_members=alumni_people_page_list,
)

##### Functions to extract data about an individual member

In [25]:
def try_load_json_file(file_path):
    """
    Attempt to load JSON data from a file.

    Parameters
    ----------
    file_path : str
        The path to the JSON file.

    Returns
    -------
    dict or None
        Returns a dictionary containing the parsed JSON data if successful,
        otherwise returns None if an error occurs or the file is inaccessible.
    """
    try:
        with open(file_path, "r") as file:
            return json.load(file)
    except Exception:
        return None

def load_individual_member_data(member_id):
    """
    Load data for an individual member based on their unique ID.

    Parameters
    ----------
    member_id : str
        Unique identifier for the member.

    Returns
    -------
    tuple
        A tuple containing:
        - dict: Basic information about the member loaded from 'info.json'.
        - dict: Dictionary containing various categories of member data loaded from respective JSON files.
            The keys correspond to categories mapped in INDIVIDUAL_MEMBER_SECTION_MAP,
            and values are dictionaries containing data for each category.
    """
    member_dir = MEMBERS_DIR_PATH / member_id
    member_jsons_dir = member_dir / "jsons"

    basic_info = try_load_json_file(member_dir / "info.json")

    member_all_data = {}
    for category in INDIVIDUAL_MEMBER_SECTION_MAP:
        member_all_data[category] = try_load_json_file(member_jsons_dir / f"{category}.json")
    
    return basic_info, member_all_data

In [26]:
columns = ["people_involved_ids", "category", "date", "title", "article_id"]
content_df = load_content_from_files(columns)

for person_id, person_data in all_people_data.items():
    basic_info, member_all_data = load_individual_member_data(person_id)
    create_page(
        "individual_person.html.j2",
        f"members/{person_id}/{person_id}.html",
        general=data["general"],
        member_id=person_id,
        member_data=person_data,
        basic_info=basic_info,
        category_data=member_all_data,
        section_headings=INDIVIDUAL_MEMBER_SECTION_MAP,
        content=content_df.to_dict(orient='records')
    )

### Contact Page

In [27]:
create_page(
    "contact.html.j2",
    "Contact.html",
    general=data["general"],
    contact=data["contact"]
)

### Support Page

In [28]:
create_page(
    "support.html.j2",
    "Support.html",
    general=data["general"],
    support=data["support"]
)

### Research Front Page

For adding more columns in dataframe to render front pages and individual article pages

In [29]:
columns_extended = ARTICLE_METADATA_FIELDS + ["author_id"]
content_df = load_content_from_files(columns_extended)
research_content_df = content_df[content_df["category"] != "News"].sort_values(
    by=["category", "date"], ascending=[True, False]
)
latest_content_df = get_latest_content_df(content_df)

In [30]:
research_content_df

,article_id,category,date,tags,title,cover_image,short_description,author_id
3,snr0509_josh_paper,Astrophysical Transients,2023-05-28,"[research, news]",A comprehensive SN Ia companion search in SNR ...,website_files/images/article_content/snr0509_v...,A search for a surviving companion to a 400 ye...,josh_shields
0,sn1006_josh_paper,Astrophysical Transients,2022-07-10,"[research, news]",Searching for a Hypervelocity White Dwarf SN I...,website_files/images/article_content/sn1006_vi...,Testing a promising SN Ia progenitor channel b...,josh_shields
4,midsure22_poster_bea,Computational Metascience,2022-07-22,[research],MIDSURE 2022,website_files/images/article_content/bea_midsu...,Poster presentation at the Mid-Michigan Sympos...,bea_lu
2,uuraf21_poster_vicente,Computational Metascience,2021-04-19,[research],MSU UURAF 2021,website_files/images/article_content/MAST_Post...,Poster presentation for MSU's University Under...,vicente_amado


In [31]:
create_page(
    "research.html.j2",
    "Research.html",
    general=data["general"],
    content=research_content_df,
    research=data["research"],
    current_members=current_people_page_list
)

In [32]:
folder_path = Path(HOSTING_PATH) / "sub_research"
folder_path.mkdir(parents=True, exist_ok=True)

for category in content_df.loc[content_df.category != "News", "category"].unique():
    create_page(
        "sub_research_frontpage.html.j2",
        f"sub_research/{page_link(category.lower())}.html",
        general=data["general"],
        research=data["research"],
        content=research_content_df,
        category=category,
        current_members=current_people_page_list
    )

Individual Research Page


In [33]:
columns_extended = ARTICLE_METADATA_FIELDS + ["author_id", "people_involved_ids", "links", "content", "long_description"]
content_df = load_content_from_files(columns_extended)
ind_research_content_df = content_df[content_df["category"] != "News"].sort_values(
    by=["category", "date"], ascending=[True, False]
)


for ind_research_keys, ind_research_values in ind_research_content_df.iterrows():
    
    folder_path = Path(HOSTING_PATH) / "sub_research" / page_link(ind_research_values.category.lower())
    folder_path.mkdir(parents=True, exist_ok=True)
    create_page(
        "research_page_no_twitter.html.j2",
        f"sub_research/{page_link(ind_research_values.category.lower())}/{page_link(ind_research_values.article_id.lower())}.html",
        general=data["general"],
        content=ind_research_values,
        member_data=all_people_data,
        article_id=ind_research_values['article_id']
    )

### News Page

In [34]:
columns_extended = ARTICLE_METADATA_FIELDS + ["author_id", "people_involved_ids", "content", "long_description"]
content_df = load_content_from_files(columns_extended)

news_content_df = content_df[
    (content_df["category"] == "News") | (content_df["tags"].apply(lambda x: "news" in x if isinstance(x, list) else False))
].sort_values(by=["date"], ascending=[False])

create_page(
    "news.html.j2",
    "News.html",
    general=data["general"],
    research=data["research"],
    content=news_content_df,
    member_data=all_people_data,
    category="News"
)

Individual News Page

In [35]:
for ind_news_keys, ind_news_values in news_content_df.iterrows():

    folder_path = Path(HOSTING_PATH) / "news" / page_link(ind_news_values.article_id.lower())
    folder_path.mkdir(parents=True, exist_ok=True)
    create_page(
        "news_page_no_twitter.html.j2",
        f"news/{page_link(ind_news_values.article_id.lower())}.html",
        general=data["general"],
        content=ind_news_values,
        member_data=all_people_data,
        category="News"
    )